In [1]:
!pip install torch

In [3]:
!pip install torchvision

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.1/1.2 MB 2.4 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.2 MB 3.1 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 4.0 MB/s eta 0:00:01
   -------------------------- ------------- 0.8/1.2 MB 4.1 MB/s eta 0:00:01
   ---------------------------------- ----- 1.0/1.2 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 4.4 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [5]:
# 데이터셋 불러오기 및 전처리
transform = transforms.Compose([transforms.ToTensor(),  # 데이터를 텐서로 변환
                                transforms.Normalize((0.5,), (0.5,))])  # 데이터를 0.5의 평균과 0.5의 표준편차로 정규화

In [6]:
# 훈련 데이터셋과 테스트 데이터셋 다운로드 및 로드
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:16<00:00, 592752.78it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 152401.27it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:03<00:00, 506638.63it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [7]:
# 데이터로더를 사용하여 데이터셋을 배치 단위로 로드
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [8]:
train_dataset[0]

(tensor([[[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,
           -1.0000, -1.0000, -1.000

In [9]:
# 신경망 모델 정의
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # 입력층: 28x28 이미지를 1차원 배열로 변환, 128개의 뉴런
        self.fc2 = nn.Linear(128, 10)  # 출력층: 10개의 뉴런 (0~9 클래스)

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # 이미지를 1차원 배열로 변환
        x = F.relu(self.fc1(x))  # 은닉층: ReLU 활성화 함수 적용
        x = self.fc2(x)  # 출력층
        return F.log_softmax(x, dim=1)  # 소프트맥스 함수로 클래스 확률 반환

In [10]:
# 모델 초기화
model = SimpleNN()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()  # 교차 엔트로피 손실 함수
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # SGD 옵티마이저

In [11]:
# 모델 학습 함수 정의
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # 모델을 학습 모드로 설정
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # 데이터를 장치로 이동
        optimizer.zero_grad()  # 이전 기울기 초기화
        output = model(data)  # 모델 예측
        loss = criterion(output, target)  # 손실 계산
        loss.backward()  # 역전파를 통해 기울기 계산
        optimizer.step()  # 가중치 업데이트
        if batch_idx % 100 == 0:  # 100번째 배치마다 로그 출력
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [12]:
# 모델 평가 함수 정의
def test(model, device, test_loader):
    model.eval()  # 모델을 평가 모드로 설정
    test_loss = 0
    correct = 0
    with torch.no_grad():  # 평가 시에는 기울기를 계산하지 않음
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # 손실 합산
            pred = output.argmax(dim=1, keepdim=True)  # 가장 높은 확률을 가진 클래스 예측
            correct += pred.eq(target.view_as(pred)).sum().item()  # 맞춘 개수 합산

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({test_accuracy:.0f}%)\n')
    return test_loss, test_accuracy

In [13]:
# 학습 및 평가
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 1
train_losses, test_losses, test_accuracies = [], [], []

In [14]:
# 모델 학습
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test_loss, test_accuracy = test(model, device, test_loader)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320575
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.622711
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.341780
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.227354
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.169311
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.180965
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.363811
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.316420
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.237704
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.153868

Test set: Average loss: 0.0002, Accuracy: 9417/10000 (94%)



In [16]:
!pip install torchsummary

In [17]:
from torchsummary import summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]         100,480
            Linear-2                   [-1, 10]           1,290
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.39
Estimated Total Size (MB): 0.39
----------------------------------------------------------------
